In [1]:
import os
import os.path
import pandas as pd
import sys
sys.path.append('..')
from src.utils.data import writePandas, getPandas
os.chdir('../..')

In [2]:
data = getPandas('pat_data')

In [18]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype import Function
import nipype.interfaces.fsl as fsl
fsl.FSLCommand.set_default_output_type('NIFTI')
import nipype.interfaces.utility as util
from nipype.interfaces.fsl import BET, FAST, ApplyMask
from nipype.interfaces.ants import RegistrationSynQuick
from nipype.interfaces.spm import Smooth
import nipype.interfaces.io as nio

In [19]:
def gm_extract(pve_files):
    return pve_files[1]

def gz2nii(gzfile):
    import nibabel as nib
    img = nib.load(gzfile)
    nii_file = gzfile.replace('.nii.gz', '.nii')
    nib.save(img, nii_file)
    return nii_file

In [42]:
key_list = data['KEY'].tolist()[:2]

wf = Workflow(name='preproc', base_dir=os.path.abspath('tmp'))

info_src = Node(util.IdentityInterface(fields=['key']), name='info_src')
info_src.iterables = ('key', key_list)

raw_src = Node(nio.DataGrabber(infields=['key'], outfields=['raw']), name='raw_src')
raw_src.inputs.base_directory = os.path.abspath(os.path.join('data', 'subj'))
raw_src.inputs.sort_filelist = False
raw_src.inputs.template = '*'
raw_src.inputs.template_args = {
    'raw': [['key']]
}
raw_src.inputs.field_template = {
    'raw': os.path.join('%s', 'raw', 'raw.nii')
}

bet = Node(BET(), name='bet')
bet.inputs.robust = True

reg = Node(RegistrationSynQuick(), name='reg')
reg.inputs.fixed_image = os.path.abspath(os.path.join('data', 'bin', 'template.nii.gz'))
reg.inputs.num_threads = 16

fslseg = Node(FAST(), name='fslseg')
fslseg.inputs.output_type = 'NIFTI'
fslseg.inputs.segments = True
fslseg.inputs.probability_maps = True
fslseg.inputs.number_classes = 3

gmextract = Node(Function(input_names=['pve_files'], output_names=['gm_file'], function=gm_extract), name='gmextract')

smooth = Node(Smooth(), name='smooth')
smooth.inputs.fwhm = 4

msk = Node(ApplyMask(), name='msk')
msk.inputs.mask_file = os.path.abspath(os.path.join('data', 'bin', 'mask.nii'))

sinker = Node(nio.DataSink(infields=['key']), name='sinker')
sinker.inputs.base_directory = os.path.abspath(os.path.join('data', 'subj'))
# bet_replace = [('_bet'+str(i), os.path.join(key_list[i], 'preproc', 'fsl')) for i in range(len(key_list))]
# reg_replace = [('_reg'+str(i), os.path.join(key_list[i], 'preproc', 'fsl')) for i in range(len(key_list))]
# fslseg_replace = [('_fslseg'+str(i), os.path.join(key_list[i], 'preproc', 'fsl')) for i in range(len(key_list))]
# msk_replace = [('_msk'+str(i), os.path.join(key_list[i], 'preproc', 'fsl')) for i in range(len(key_list))]
# substitutions = [
#     ('raw_brain', 'brain'),
#     ('transformWarped', 'reg'),
#     ('pve_0', 'csf'),
#     ('pve_1', 'gm'),
#     ('pve_2', 'wm'),
# ]
# sinker.inputs.regexp_substitutions = bet_replace + reg_replace + fslseg_replace + msk_replace + substitutions
sinker.inputs.regexp_substitutions = [
    ('raw_brain', 'brain'),
    ('transformWarped', 'reg'),
    ('pve_0', 'csf'),
    ('pve_1', 'gm'),
    ('pve_2', 'wm'),
    ('_key_', ''),
]

wf.connect([
    (info_src, raw_src, [('key', 'key')]),
    (raw_src, bet, [('raw', 'in_file')]),
    (bet, reg, [('out_file', 'moving_image')]),
    (reg, fslseg, [('warped_image', 'in_files')]),
    (fslseg, gmextract, [('partial_volume_files', 'pve_files')]),
    (gmextract, smooth, [('gm_file', 'in_files')]),
    (smooth, msk, [('smoothed_files', 'in_file')]),
    #(info_src, sinker, [('key', 'container')]),
    (bet, sinker, [('out_file', '@out_file')]),
    (reg, sinker, [('warped_image', '@warped_image')]),
    (fslseg, sinker, [('partial_volume_files', '@partial_volume_files')]),
    (msk, sinker, [('out_file', '@masked_smoothed_file')]),
])

In [37]:
key_list

['3107V04I296431', '3107V06I378218']

In [43]:
wf.run(plugin='MultiProc', plugin_args={'n_procs': 16})

230331-12:21:11,864 nipype.workflow INFO:
	 Workflow preproc settings: ['check', 'execution', 'logging', 'monitoring']
230331-12:21:11,896 nipype.workflow INFO:
	 Running in parallel.
230331-12:21:11,900 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 26.21/26.21, Free processors: 16/16.
230331-12:21:12,54 nipype.workflow INFO:
	 [Node] Setting-up "preproc.raw_src" in "/home/biobot/disk/ldopa_pred/tmp/preproc/_key_3107V06I378218/raw_src".
230331-12:21:12,56 nipype.workflow INFO:
	 [Node] Setting-up "preproc.raw_src" in "/home/biobot/disk/ldopa_pred/tmp/preproc/_key_3107V04I296431/raw_src".
230331-12:21:12,58 nipype.workflow INFO:
	 [Node] Executing "raw_src" <nipype.interfaces.io.DataGrabber>
230331-12:21:12,60 nipype.workflow INFO:
	 [Node] Finished "raw_src", elapsed time 0.000328s.
230331-12:21:12,60 nipype.workflow INFO:
	 [Node] Executing "raw_src" <nipype.interfaces.io.DataGrabber>
230331-12:21:12,62 nipype.workflow INFO:
	 [Node] Finished